<a href="https://colab.research.google.com/github/GianlucaRapaglia/LLM-training/blob/main/01%20-%20LLM%20validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Evaluate Fine-Tuned GPT-2 Model on WikiText-2

This notebook demonstrates how to validate a **fine-tuned DistilGPT-2 model** on the dataset it was trained on: `wikitext/wikitext-2-raw-v1`.

We will:
- Load the model and tokenizer from Hugging Face Hub
- Load and preprocess the test set of WikiText-2
- Evaluate the model using **perplexity**, a standard metric in language modeling


In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
